In [1]:
import os
from dotenv import load_dotenv, find_dotenv

In [2]:
# find .env automagically by walking up directories until it's found
dotenv_path = find_dotenv()

# load up the entries as environment variables
load_dotenv(dotenv_path)

aws_access_key = os.environ.get("AWS_ACCESS_KEY")
aws_secret_access_key = os.environ.get("AWS_SECRET_ACCESS_KEY")
aws_bucket = os.environ.get("AWS_BUCKET")
root_dir = os.environ.get("ROOT_DIR")

In [3]:
import boto3

In [4]:
# Initialize the session with authentication
session = boto3.Session(
    aws_access_key_id=aws_access_key,
    aws_secret_access_key=aws_secret_access_key,
)
 
# Get the interface to AWS s3
#s3_client = session.client(
#    "s3", endpoint_url="https://s3.wasabisys.com"
#)

s3_resource = session.resource("s3", endpoint_url = 'https://s3.wasabisys.com')
s3_bucket = s3_resource.Bucket(aws_bucket)

In [5]:
paths = ('core/', 'geo-supplement/', 'monthly-patterns/normalization_stats', 'monthly-patterns/patterns', 
        'social-distancing/v2/')

In [ ]:
# local directory where we want to put all the data
local = os.path.join(root_dir,'data/raw')
n = len(local)
# print(local)
files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(local):
    for file in f:
        files.append(os.path.join(r, file)[n+1:])
#for file in files:
#    print(file)
count = 0
for obj in s3_bucket.objects.all():
    count = count + 1
    file = obj.key
    if file.startswith(paths) and file not in files:
        print(file)
        filepath = os.path.join(local, file)
        directory = os.path.dirname(filepath)
        os.makedirs(directory, exist_ok=True)
        s3_bucket.download_file(file, filepath)

monthly-patterns/patterns/2020/10/07/02/patterns-part2.csv.gz
monthly-patterns/patterns/2020/10/07/02/patterns-part3.csv.gz
